# Merge household income onto satellite data

General steps:
1. Import data
2. Prep BISP household-year level data for merge
3. Merge onto satellite data
4. Export to CSV

To do:
1. Merge on province/district/tehsil/UC names for each household

In [7]:
import os
import numpy as np
import pandas as pd 

# Display options 
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = -1

# Data file path 
final_data_file_path = "/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data/FinalData"
#

### 1. Import data

In [8]:
# Import household income data
BISP_PATH = os.path.join(final_data_file_path, 'Outputs for Analysis TEMP', 'bisp_hh_income.csv')
bisp_df = pd.read_csv(BISP_PATH)
bisp_df.shape

(33794, 4)

In [9]:
bisp_df.head()

,uid,period,hh_inc,pscores
0,100215,2011,9999998,18.080000
1,100215,2013,240000,18.080000
2,100215,2014,0,30.059999
3,100215,2016,0,31.240000
4,100385,2016,0,10.300000


In [10]:
# Import satellite imagery data
SAT_PATH = os.path.join(final_data_file_path, 'Outputs for Analysis TEMP', 'bisp_satellite_data.csv')
sat_df = pd.read_csv(SAT_PATH)
sat_df.shape

(5416, 44)

In [11]:
sat_df.head()

,uid,viirs_2012,viirs_2013,viirs_2014,viirs_2015,viirs_2016,viirs_2017,viirs_2018,dmspols_1992,dmspols_1993,dmspols_1994,dmspols_1995,dmspols_1996,dmspols_1997,dmspols_1998,dmspols_1999,dmspols_2000,dmspols_2001,dmspols_2002,dmspols_2003,dmspols_2004,dmspols_2005,dmspols_2006,dmspols_2007,dmspols_2008,dmspols_2009,dmspols_2010,dmspols_2011,dmspols_2012,dmspols_2013,l7_2011_1,l7_2011_2,l7_2011_3,l7_2011_4,l7_2011_5,l7_2011_6,l7_2011_7,l7_2013_1,l7_2013_2,l7_2013_3,l7_2013_4,l7_2013_5,l7_2013_6,l7_2013_7
0,100389,2.052018,2.141392,2.089507,2.307763,2.850603,3.653005,3.750000,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,902.331348,1224.739396,1393.123911,2555.792708,2474.174317,3005.856769,1922.539802,951.897734,1282.748257,1417.251598,2574.000436,2469.137711,3000.408919,1889.609384
1,100401,1.964332,2.133366,2.052437,2.296554,2.769960,3.702374,3.488333,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,885.841488,1200.548350,1366.253764,2512.672843,2451.849595,3004.616242,1890.566155,941.063694,1268.392009,1402.777070,2547.212362,2463.117111,2998.701940,1876.871453
2,100581,1.824753,1.937131,1.875487,2.047540,2.557241,3.198625,3.286000,43.0,32.500000,34.25,43.000000,38.0,31.750000,38.25,38.750000,36.000000,38.250000,37.750000,32.000000,32.75,33.75,40.000000,43.75,42.5,30.000000,45.5,30.500000,47.500000,44.500000,886.021385,1206.745127,1373.031277,2550.999418,2462.909660,3006.164678,1900.649840,935.162060,1263.157696,1398.079866,2572.847832,2458.750073,2999.056008,1880.909223
3,101101,1.964332,2.133366,2.052437,2.296554,2.769960,3.702374,3.488333,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,886.196798,1201.037263,1366.468559,2514.479913,2450.865939,3004.699563,1890.108734,940.979913,1268.248763,1401.871616,2547.740466,2459.946143,2998.786463,1874.074672
4,101236,2.052018,2.141392,2.089507,2.307763,2.850603,3.653005,3.750000,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,891.264553,1209.613090,1374.709528,2535.919345,2453.881552,3005.134086,1897.493484,943.113959,1271.824645,1403.386186,2563.689111,2453.279467,2999.725311,1869.047929


### 2. Transform BISP household data

In [12]:
# Only keep years with income data
bisp_df = bisp_df.loc[(bisp_df['period'] == 2011) | (bisp_df['period'] == 2013)]

bisp_df.head()

,uid,period,hh_inc,pscores
0,100215,2011,9999998,18.080000
1,100215,2013,240000,18.080000
5,100389,2011,9000,23.370001
6,100389,2013,73000,29.889999
8,100401,2011,75000,5.110000


In [13]:
# Reshape wide
bisp_df = bisp_df.pivot(index='uid', columns='period', values='pscores') \
    .reset_index() \
    .rename({2011: 'pscores_2011', 2013: 'pscores_2013'}, axis=1)

bisp_df.head()

period,uid,pscores_2011,pscores_2013
0,100215,18.080000,18.080000
1,100389,23.370001,29.889999
2,100401,5.110000,2.650000
3,100581,4.300000,5.510000
4,101000,0.000000,5.900000


### 3. Merge with satellite data

In [14]:
sat_df.shape

(5416, 44)

In [15]:
sat_df = sat_df.merge(right=bisp_df, how='left', on='uid')

sat_df.head()

,uid,viirs_2012,viirs_2013,viirs_2014,viirs_2015,viirs_2016,viirs_2017,viirs_2018,dmspols_1992,dmspols_1993,dmspols_1994,dmspols_1995,dmspols_1996,dmspols_1997,dmspols_1998,dmspols_1999,dmspols_2000,dmspols_2001,dmspols_2002,dmspols_2003,dmspols_2004,dmspols_2005,dmspols_2006,dmspols_2007,dmspols_2008,dmspols_2009,dmspols_2010,dmspols_2011,dmspols_2012,dmspols_2013,l7_2011_1,l7_2011_2,l7_2011_3,l7_2011_4,l7_2011_5,l7_2011_6,l7_2011_7,l7_2013_1,l7_2013_2,l7_2013_3,l7_2013_4,l7_2013_5,l7_2013_6,l7_2013_7,pscores_2011,pscores_2013
0,100389,2.052018,2.141392,2.089507,2.307763,2.850603,3.653005,3.750000,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,902.331348,1224.739396,1393.123911,2555.792708,2474.174317,3005.856769,1922.539802,951.897734,1282.748257,1417.251598,2574.000436,2469.137711,3000.408919,1889.609384,23.370001,29.889999
1,100401,1.964332,2.133366,2.052437,2.296554,2.769960,3.702374,3.488333,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,885.841488,1200.548350,1366.253764,2512.672843,2451.849595,3004.616242,1890.566155,941.063694,1268.392009,1402.777070,2547.212362,2463.117111,2998.701940,1876.871453,5.110000,2.650000
2,100581,1.824753,1.937131,1.875487,2.047540,2.557241,3.198625,3.286000,43.0,32.500000,34.25,43.000000,38.0,31.750000,38.25,38.750000,36.000000,38.250000,37.750000,32.000000,32.75,33.75,40.000000,43.75,42.5,30.000000,45.5,30.500000,47.500000,44.500000,886.021385,1206.745127,1373.031277,2550.999418,2462.909660,3006.164678,1900.649840,935.162060,1263.157696,1398.079866,2572.847832,2458.750073,2999.056008,1880.909223,4.300000,5.510000
3,101101,1.964332,2.133366,2.052437,2.296554,2.769960,3.702374,3.488333,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,886.196798,1201.037263,1366.468559,2514.479913,2450.865939,3004.699563,1890.108734,940.979913,1268.248763,1401.871616,2547.740466,2459.946143,2998.786463,1874.074672,8.190000,22.400000
4,101236,2.052018,2.141392,2.089507,2.307763,2.850603,3.653005,3.750000,43.0,33.666667,35.50,45.333333,40.0,33.166667,39.50,40.333333,37.333333,39.666667,38.833333,33.666667,34.00,34.50,40.666667,45.00,43.0,30.333333,46.0,32.666667,47.666667,45.333333,891.264553,1209.613090,1374.709528,2535.919345,2453.881552,3005.134086,1897.493484,943.113959,1271.824645,1403.386186,2563.689111,2453.279467,2999.725311,1869.047929,40.750000,NaN


In [16]:
sat_df.shape 
# No rows lost

(5416, 46)

### 4. Export to csv

In [17]:
OUTPUT_PATH = os.path.join(final_data_file_path, 'Outputs for Analysis TEMP', 'bisp_sat_pov_data.csv')
sat_df.to_csv(OUTPUT_PATH, index=False)